# 1.1.	Подготовка данных для работы

In [23]:
import cv2 #импорт библиотеки cv2



cap = cv2.VideoCapture('./bicycle.mp4') #созданиие объекта VideoCapture
count = 0 # инициализация переменной-счетчика
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # вычисление длинны видеоряда



while cap.isOpened(): #цикл раскадровки
    ret,frame = cap.read() # запись кадара в frame
    cv2.imshow('Bicycle',frame) #отображение всех кадров
    cv2.imwrite("./frames_of_video/frame%d.jpg" % count, frame)#запись всех кадров
    
    count = count + 1
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
    
    if count == length: #проверка на выполнение
        break
        

        
cap.release()
cv2.destroyWindow('Red Ball') #закрытие окна

# 1.2.	Алгоритмы обнаружения объекта OpenCV

In [ ]:
import cv2
import video
import numpy as np

if __name__ == '__main__':
    
    def nothing(*arg):
        pass
    
    cv2.namedWindow("original")
    
    cap = video.create_capture(0)
    
    kernel = np.array([[-0.1, -0.1, -0.1], [-1, 5, 1], [-0.1, -0.1, -0.1]])
    
    while True:
        flag, img = cap.read()
        
        low_blue = np.array((13, 153, 153), np.uint8)
        high_blue = np.array((18, 255, 255), np.uint8)
        try:
            #dst = cv2.filter2D(img, -7, kernel) выцветание
            #dst = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) помехи
            img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            mask_blue = cv2.inRange(img_hsv, low_blue, high_blue)
            
            #result = cv2.bitwise_and(img_hsv, img_hsv, mask = mask_blue)
            #result = cv2.cvtColor(result, cv2.COLOR_HSV2BGR)
            
            moments = cv2.moments(mask_blue, 1)
            
            dM01 = moments['m01']
            dM10 = moments['m10']
            dArea = moments['m00']
            
            if dArea > 150:
                x = int(dM10 / dArea)
                y = int(dM01 / dArea)
                cv2.circle(img, (x, y), 10, (255, 0, 0), -1)
            
            cv2.imshow("original", img)
        except:
            cap.release()
            raise
        
        ch = cv2.waitKey(5)
        
        if ch == 27:
            break
            
    cap.release()
    cv2.destroyAllWindows()
    

In [8]:
!pip install imutils

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
'''
Object detection ("Ball tracking") with OpenCV
    Adapted from the original code developed by Adrian Rosebrock
    Visit original post: https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/
Developed by Marcelo Rovai - MJRoBot.org @ 7Feb2018 
'''

# import the necessary packages
from collections import deque
import numpy as np
import argparse
import imutils
import cv2

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video",
	help="path to the (optional) video file")
ap.add_argument("-b", "--buffer", type=int, default=64,
	help="max buffer size")
args = vars(ap.parse_args())

# define the lower and upper boundaries of the "yellow object"
# (or "ball") in the HSV color space, then initialize the
# list of tracked points
colorLower = (24, 100, 100)
colorUpper = (44, 255, 255)
pts = deque(maxlen=args["buffer"])
 
# if a video path was not supplied, grab the reference
# to the webcam
if not args.get("video", False):
	camera = cv2.VideoCapture(0)
 
# otherwise, grab a reference to the video file
else:
	camera = cv2.VideoCapture(args["video"])

# keep looping
while True:
    # grab the current frame
    (grabbed, frame) = camera.read()
 
    # if we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
    if args.get("video") and not grabbed:
        break
 
    # resize the frame, inverted ("vertical flip" w/ 180degrees),
    # blur it, and convert it to the HSV color space
    frame = imutils.resize(frame, width=600)
    frame = imutils.rotate(frame, angle=180)
    # blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
 
    # construct a mask for the color "green", then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # find contours in the mask and initialize the current
    # (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
 
    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
 
        # only proceed if the radius meets a minimum size
        if radius > 10:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
 
    # update the points queue
    pts.appendleft(center)
    
        # loop over the set of tracked points
    for i in range(1, len(pts)):
        # if either of the tracked points are None, ignore
        # them
        if pts[i - 1] is None or pts[i] is None:
            continue
 
        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        thickness = int(np.sqrt(args["buffer"] / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
 
    # show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
 
    #cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

usage: ipykernel_launcher.py [-h] [-v VIDEO] [-b BUFFER]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Вячеслав\AppData\Roaming\jupyter\runtime\kernel-f84d3f6c-93a2-4b8e-99a7-807ce71c94ca.json


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
import cv2
import numpy as np

def getthresholdedimg(hsv):
    yellow = cv2.inRange(hsv,np.array((20,100,100)),np.array((30,255,255)))
    blue = cv2.inRange(hsv,np.array((100,100,100)),np.array((120,255,255)))
    both = cv2.add(yellow,blue)
    return both

c = cv2.VideoCapture(0)
width,height = c.get(3),c.get(4)
print ("frame width and height : ", width, height)

while(1):
    _,f = c.read()
    f = cv2.flip(f,1)
    blur = cv2.medianBlur(f,5)
    hsv = cv2.cvtColor(f,cv2.COLOR_BGR2HSV)
    both = getthresholdedimg(hsv)
    erode = cv2.erode(both,None,iterations = 3)
    dilate = cv2.dilate(erode,None,iterations = 10)

    contours,hierarchy = cv2.findContours(dilate,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        cx,cy = x+w/2, y+h/2
        
        if 20 < hsv.item(cy,cx,0) < 30:
            cv2.rectangle(f,(x,y),(x+w,y+h),[0,255,255],2)
            print ("yellow :", x,y,w,h)
        elif 100 < hsv.item(cy,cx,0) < 120:
            cv2.rectangle(f,(x,y),(x+w,y+h),[255,0,0],2)
            print ("blue :", x,y,w,h)

    cv2.imshow('img',f)

    if cv2.waitKey(25) == 27:
        break

cv2.destroyAllWindows()
c.release()

frame width and height :  640.0 480.0


TypeError: 'float' object cannot be interpreted as an integer

In [2]:
!pip install tensorflow

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
!pip install numpy

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!pip install scipy

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!pip install opencv-python

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!pip install pillow

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
!pip install matplotlib

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
!pip install h5py

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
!pip install keras

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
!pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
!pip install imageai

  Using cached https://files.pythonhosted.org/packages/09/99/4023e191a343fb23f01ae02ac57a5ca58037c310e8d8c62f87638a3bafc7/imageai-2.1.5-py3-none-any.whl


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "image.jpg"), output_image_path=os.path.join(execution_path , "imagenew.jpg"))

for eachObject in detections:
    print(eachObject["name"] , " : " , eachObject["percentage_probability"] )

car  :  87.97370791435242
car  :  61.55836582183838
car  :  93.1560754776001
car  :  96.8542993068695
car  :  96.94370031356812
car  :  89.06227350234985
car  :  87.87435293197632
car  :  93.10488700866699
car  :  99.0717887878418
car  :  97.95353412628174
car  :  95.81040143966675
car  :  97.74502515792847
car  :  93.26549768447876


In [6]:
import os.path
path = 'frames_of_video'
num_files = len([f for f in os.listdir(path)
                if os.path.isfile(os.path.join(path, f))])

In [14]:
image_arr = list(range(2200, num_files+2200))

In [8]:
print(os.path.join(execution_path , "frames_of_video\\frame1d.jpg"))

C:\Users\Вячеслав\source\repos\objectDetection\frames_of_video\frame1d.jpg


In [15]:
for i in image_arr:
    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "frames_of_video\\frame%d.jpg" % i), output_image_path=os.path.join(execution_path , "result\\imagenew%d.jpg" % i))

In [18]:
import cv2
import numpy as np
try:
    from pathlib import Path
except ImportError:             # Python 2
    from pathlib2 import Path

In [ ]:
p = Path('frames_of_video')

#читаем кадры
imgs = [cv2.imread(str(f)) for f in p.glob('*.jpg')]

# создаем видео
height, width, layers = imgs[0].shape
video = cv2.VideoWriter(r'd:/temp/video.avi',-1,1,(width,height))
_ = [video.write(i) for i in imgs]

cv2.destroyAllWindows()
video.release()

In [20]:

p = Path('result')
p


WindowsPath('result')

In [21]:

#читаем кадры
imgs = [cv2.imread(str(f)) for f in p.glob('*.jpg')]

In [31]:
# создаем видео
height, width, layers = imgs[0].shape
video = cv2.VideoWriter(r'video.avi',-1,1,(width,height))
_ = [video.write(i) for i in imgs]

cv2.destroyAllWindows()
video.release()

# 1.3.	Нейросетевые алгоритмы детектирования объектов.

In [39]:
cv2.VideoWriter(r'video.avi',-1,1,(width,height))

<VideoWriter 000002883F5FBF70>

# 1.4.	Оценки качества работы алгоритмов